In [1]:
import pandas as pd
from scrapy.http import TextResponse
import requests
import datetime

In [2]:
title_ls,link_ls = [],[]
content_str = 0
for page in range(1,30+1):
    url = 'https://m.news.naver.com/newsflash.nhn?mode=LS2D&sid1=101&sid2=259&page={}'.format(page)
    req = requests.get(url)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    for content in range(1,20+1):
        try: 
            content_str += 1
            title_ls.append(response.xpath('//*[@id="newsflash{}"]/a/div/span[1]/strong/text()'.format(content_str)).extract()[0])
            link_ls.append(response.xpath('//*[@id="newsflash{}"]/a/@href'.format(content_str)).extract()[0])
        except : print(page,content)

In [3]:
len(title_ls) == len(link_ls)

True

In [4]:
df = pd.DataFrame()
df['title'] = title_ls
df['link'] = link_ls
df['link'] = ['https://m.news.naver.com' + i for i in df['link']]
df.tail()

,title,link
595,내년 초 車 보험료 3% 오른다,https://m.news.naver.com/read.nhn?oid=015&aid=...
596,"우체국, 포스트페이 실적 따라 금리 더 주는 예금 내놓는다",https://m.news.naver.com/read.nhn?oid=015&aid=...
597,적격대출도 집값만큼만 상환,https://m.news.naver.com/read.nhn?oid=015&aid=...
598,외국인은 사용할 수 없는 한국형 QR코드,https://m.news.naver.com/read.nhn?oid=015&aid=...
599,"농협금융, 베트남 최대은행과 협력 논의",https://m.news.naver.com/read.nhn?oid=015&aid=...


In [6]:
content_ls = []

for i in df['link'].values:
    req = requests.get(i)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    content_ls.append(','.join(response.xpath('//*[@id="dic_area"]/text()').extract()).replace(',',''))

In [7]:
len(title_ls) == len(link_ls) == len(content_ls)

True

In [8]:
df['content'] = content_ls
df['content'] = [i.replace("\t",'').replace('\n','') for i in df['content'].values]
df.tail()

,title,link,content
595,내년 초 車 보험료 3% 오른다,https://m.news.naver.com/read.nhn?oid=015&aid=...,[ 서정환 기자 ] 메리츠화재를 시작으로 주요 손해보험사들이 연내 자동차 보험료를...
596,"우체국, 포스트페이 실적 따라 금리 더 주는 예금 내놓는다",https://m.news.naver.com/read.nhn?oid=015&aid=...,[ 송형석 기자 ] 우정사업본부가 스마트폰으로 결제하는 포스트페이 서비스 이용실적...
597,적격대출도 집값만큼만 상환,https://m.news.naver.com/read.nhn?oid=015&aid=...,[ 박신영 기자 ] 주택금융공사의 적격대출에도 집값만큼만 빚을 책임지는 유한책임(...
598,외국인은 사용할 수 없는 한국형 QR코드,https://m.news.naver.com/read.nhn?oid=015&aid=...,[ 김순신 기자 ] 정부가 발표한 QR코드 표준으로 인해 ‘한국형 QR코드’는 외...
599,"농협금융, 베트남 최대은행과 협력 논의",https://m.news.naver.com/read.nhn?oid=015&aid=...,농협금융은 지난 9일 서울 서대문 본사에서 베트남 최대 은행인 아그리뱅크와 협력사업...


In [9]:
df.to_csv('{}_with_link_naver.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False),index=False)